## Alaska Berry Futures
### Wind speed and pollinators

February 16, 2023    
Using SNAP wind models to investigate how wind speed my affect pollinator movements in AK's future    

In [1]:
import requests
import csv
import requests
import pandas as pd
import os
from io import StringIO

In [2]:
working_directory = os.getcwd()
weather_stations =  pd.read_csv(working_directory + "\\SNAPwind_location_lookup_table-1.csv")

# Group the dataframe by the 'region' column, and get the unique values of 'icao_code' for each group
# save the codes as lowercase so it matches the csv names on the SNAP website
icao_codes_by_region = weather_stations.groupby('region')['icao_code'].apply(lambda x: x.str.lower().unique())

# Print the resulting dictionary of icao_codes by region
print(icao_codes_by_region)


region
Alaska Range Transition              [paaq, paed, paen, paho, pail, pamr, panc, pas...
Aleutian Meadows                      [pacd, padk, padq, padu, paph, pasd, pasn, pasy]
Bering Taiga                                [pabe, pacz, padl, paeh, pakn, pasm, paun]
Bering Tundra                                                 [paom, paot, pash, patc]
Coastal Rainforests                  [pacv, pael, pags, pagy, pajn, pakt, pamd, pan...
Copper River & Wrangell Mountains                                   [padt, pagk, pamx]
Intermontane Boreal                  [pabi, pabt, paei, pafa, paga, paim, pamc, pam...
North Slope                                             [paba, pabr, pafb, palu, pasc]
Name: icao_code, dtype: object


In [3]:
def download_csv(url):
    """
    Downloads a CSV file from a URL and returns its contents as a Pandas DataFrame.

    :param url: The URL of the CSV file to download.
    :return: A Pandas DataFrame containing the contents of the CSV file.
    """
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    # Print an error message if the request failed
    if response.status_code != 200:
        raise ValueError(f"Failed to download CSV from {url}. Status code: {response.status_code}")
        
    # Load the content of the response into a Pandas DataFrame
    content = response.content.decode('utf-8')
    data = pd.read_csv(StringIO(content))
    
    # Return the parsed data
    return data


In [4]:
def download_csvs_for_locations(regions, model):
    """
    Download CSV data for multiple regions.

    Args:
        regions (list): A list of region names to download CSV data for.

    Returns:
        dict: A dictionary containing a Pandas DataFrame for each location.
              The location name is used as the key in the dictionary.
    """
    # URL format string
    url_format = {
        'past_asos': 'http://data.snap.uaf.edu/data/Base/AK_WRF/Historical_Projected_Hourly_Winds_1980_2099/asos/wind_hourly_asos_{}_1980-2015.csv',
        'past_wrf_era_interim': 'http://data.snap.uaf.edu/data/Base/AK_WRF/Historical_Projected_Hourly_Winds_1980_2099/wrf/wind_hourly_wrf_era_interim_{}_1980-2014.csv',
        'future_wrf_gfdl_cm3': 'http://data.snap.uaf.edu/data/Base/AK_WRF/Historical_Projected_Hourly_Winds_1980_2099/wrf/wind_hourly_wrf_gfdl_cm3_{}_2015-2099.csv',
        'future_wrf_ncar_ccsm4': 'http://data.snap.uaf.edu/data/Base/AK_WRF/Historical_Projected_Hourly_Winds_1980_2099/wrf/wind_hourly_wrf_ncar_ccsm4_{}_2015-2099.csv'
            }
    
    # Dictionary to store the data frames
    data_frames = {}

    # Loop over locations and download the CSV data
    for region in regions:
        url = url_format[model].format(region)
        data = download_csv(url)

        # Add the data frame to the dictionary using the location name as the key
        data_frames[region] = data

    # Return the dictionary of data frames
    return data_frames

In [5]:
def extract_may_june_dataframes(dataframes):
    """
    Extracts the May/June data from a dictionary of location dataframes.

    Args:
    dataframes (dict): A dictionary of location dataframes, where the keys are location names and the values are dataframes.

    Returns:
    dict: A dictionary of location dataframes containing only the May/June data.
    """
    output = {}
    for location, df in dataframes.items():
        # Convert the 'ts' column to a datetime object
        df['ts'] = pd.to_datetime(df['ts'])
        
        # Select only the data from May and June
        may_june_data = df.loc[(df['ts'].dt.month == 5) | (df['ts'].dt.month == 6)]
        
        # Store the May/June data in the output dictionary
        output[location] = may_june_data
    
    return output


In [6]:
# spring_interior = extract_may_june_dataframes(wind_interior)
# pafa = spring_interior['pafa']
# pafa[(pafa['ws']>=10)].groupby(pafa['ts'].dt.year)['ts'].count()

In [6]:
def count_high_wind_by_year(locations_dataframes):
    result = pd.DataFrame()
    
    # Loop through each location dataframe
    for location, df in locations_dataframes.items():
        # Select only the data with wind speed >= 10
        high_wind = df.loc[df['ws'] >= 10]
        
        # Group by year and count the number of rows
        count_by_year = high_wind.groupby(high_wind['ts'].dt.year)['ts'].count()
        
        # Rename the count column to the location name
        count_by_year = count_by_year.rename(location)
        
        # Merge the count column with the result dataframe
        result = pd.concat([result, count_by_year], axis=1, sort=False)
    
    # Reset the index to be the year
    result = result.reset_index()
    
    # Rename the index column to 'year'
    result = result.rename(columns={'index': 'year'})
    
    return result


In [7]:
def spring_wind_by_region(AKregion: str, weatherdf: pd.DataFrame, model: str) -> pd.DataFrame:
    """
    Return a DataFrame with the count of high wind events for each location in a region during May and June
    for each year in the dataset.
    
    Args:
    - AKregion (str): the region for which to compute the wind counts
    - weatherdf (pd.DataFrame): a DataFrame containing weather data for various locations and time periods
    - model (str): one of four SNAP wind models: 
            past_asos, past_wrf_era_interim, future_wrf_gfdl_cm3, future_wrf_ncar_ccsm4
    
    Returns:
    - pd.DataFrame: a DataFrame with the count of high wind events for each location in the region, by year
    
    """
    # group locations by region, convert to lowercase, and get unique codes
    icao_codes_by_region = weatherdf.groupby('region')['icao_code'].apply(lambda x: x.str.lower().unique())
    
    # get list of locations within specified region
    locations = icao_codes_by_region[AKregion]
    
    # download csvs for each location and get data for May and June only
    spring_winds = extract_may_june_dataframes(download_csvs_for_locations(locations, model))
    
    # count the number of high wind events for each location, by year
    location_spring_winds = count_high_wind_by_year(spring_winds)
    
    return location_spring_winds

    

In [9]:
# spring_interior = extract_may_june_dataframes(wind_IntermontaneBoreal)
# locations_dataframes = {'pafa': spring_interior['pafa'], 'pabi': spring_interior['pabi'], 'pabt': spring_interior['pabt']}
# result = count_high_wind_by_year(locations_dataframes)
# print(result)


In [10]:
#interior_past_asos = spring_wind_by_region("Intermontane Boreal", weather_stations, "past_asos")

In [11]:
#interior_past_asos = interior_past_asos.set_index('year')


In [12]:
#interior_past_asos

In [13]:
#interior_future_ncar = spring_wind_by_region("Intermontane Boreal", weather_stations, "future_wrf_ncar_ccsm4")


In [14]:
#interior_future_ncar['mean'] = interior_future_ncar.mean(axis=1)

In [15]:
#interior_future_ncar[['mean']][50:]

In [16]:
# # ALASKA RANGE

# ART_past_asos = spring_wind_by_region("Alaska Range Transition", weather_stations, "past_asos").mean(axis=1)
# ART_future_ncar = spring_wind_by_region("Alaska Range Transition", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# ART_past_interim = spring_wind_by_region("Alaska Range Transition", weather_stations, "past_wrf_era_interim").mean(axis=1)
# ART_future_gfdl_cm3 = spring_wind_by_region("Alaska Range Transition", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [8]:
# # ALEUTIAN MEADOWS

#AM_past_asos = spring_wind_by_region("Aleutian Meadows", weather_stations, "past_asos").mean(axis=1)
#AM_future_ncar = spring_wind_by_region("Aleutian Meadows", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
#AM_past_interim = spring_wind_by_region("Aleutian Meadows", weather_stations, "past_wrf_era_interim").mean(axis=1)
#AM_future_gfdl_cm3 = spring_wind_by_region("Aleutian Meadows", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [11]:
# # BERING TAIGA

# BTa_past_asos = spring_wind_by_region("Bering Taiga", weather_stations, "past_asos").mean(axis=1)
# BTa_future_ncar = spring_wind_by_region("Bering Taiga", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# BTa_past_interim = spring_wind_by_region("Bering Taiga", weather_stations, "past_wrf_era_interim").mean(axis=1)
# BTa_future_gfdl_cm3 = spring_wind_by_region("Bering Taiga", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [12]:
BTa_past_asos

0     836.714286
1     894.857143
2     922.142857
3     850.142857
4     673.375000
5     633.250000
6     697.375000
7     614.875000
8     593.375000
9     688.250000
10    617.875000
11    743.875000
12    792.571429
13    829.625000
14    755.375000
15    765.250000
16    766.000000
17    696.500000
18    779.875000
19    703.625000
20    635.875000
21    667.375000
22    754.500000
23    754.375000
24    758.375000
25    747.625000
26    801.750000
27    682.250000
28    758.250000
29    793.750000
30    744.875000
31    724.000000
32    738.875000
33    786.000000
34    792.750000
dtype: float64

In [13]:
BTa_past_interim

0     856.375
1     776.625
2     773.500
3     722.625
4     665.500
5     751.125
6     915.375
7     832.625
8     678.125
9     802.500
10    742.875
11    818.000
12    815.125
13    794.375
14    731.875
15    775.250
16    848.000
17    674.250
18    832.000
19    688.625
20    703.250
21    832.625
22    744.625
23    738.375
24    747.125
25    706.750
26    818.625
27    671.625
28    728.875
29    780.625
30    789.750
31    797.125
32    796.250
33    789.750
34    763.500
dtype: float64

In [14]:
BTa_future_ncar[0:50]

0     805.125
1     698.000
2     696.375
3     646.000
4     714.875
5     769.250
6     673.750
7     707.375
8     749.125
9     712.250
10    729.750
11    688.500
12    671.875
13    728.125
14    771.250
15    759.375
16    738.125
17    816.500
18    713.375
19    889.875
20    695.625
21    613.375
22    658.875
23    845.500
24    779.000
25    780.625
26    685.500
27    819.750
28    656.500
29    858.250
30    669.125
31    740.125
32    822.250
33    696.875
34    788.750
35    946.625
36    842.625
37    687.625
38    769.000
39    753.125
40    755.000
41    794.875
42    904.875
43    590.875
44    670.000
45    787.250
46    673.625
47    782.500
48    742.250
49    571.125
dtype: float64

In [15]:
BTa_future_ncar[50:]

50    722.875
51    794.750
52    773.875
53    678.750
54    699.250
55    768.750
56    720.375
57    646.500
58    678.625
59    869.875
60    741.125
61    734.875
62    664.125
63    720.500
64    771.875
65    639.375
66    792.125
67    792.375
68    747.500
69    618.625
70    894.875
71    855.125
72    692.375
73    880.750
74    643.500
75    727.250
76    775.375
77    783.250
78    653.250
79    772.000
80    697.750
81    606.375
82    719.125
83    797.625
84    694.375
dtype: float64

In [16]:
BTa_future_gfdl_cm3[0:50]

0     810.500
1     578.750
2     860.000
3     706.250
4     616.125
5     768.000
6     797.875
7     847.000
8     618.375
9     654.500
10    766.000
11    601.875
12    750.375
13    800.500
14    862.250
15    736.125
16    844.500
17    730.625
18    777.125
19    761.625
20    660.375
21    619.375
22    707.125
23    814.375
24    797.625
25    666.625
26    685.375
27    786.250
28    835.875
29    668.125
30    757.000
31    696.125
32    702.875
33    776.125
34    903.125
35    729.250
36    609.875
37    688.250
38    603.875
39    758.250
40    700.375
41    769.625
42    645.000
43    683.500
44    758.875
45    598.250
46    701.250
47    793.000
48    650.000
49    846.500
dtype: float64

In [17]:
BTa_future_gfdl_cm3[50:]

50    608.875
51    836.000
52    600.625
53    659.250
54    782.500
55    657.750
56    819.125
57    633.125
58    771.250
59    726.375
60    819.750
61    705.000
62    838.000
63    866.000
64    796.875
65    693.500
66    688.250
67    570.500
68    644.500
69    717.625
70    768.375
71    751.000
72    826.625
73    848.000
74    858.125
75    767.500
76    814.875
77    619.625
78    618.625
79    629.625
80    699.000
81    679.125
82    768.875
83    653.000
84    765.000
dtype: float64

In [18]:
# # BERING TUNDRA

# BTu_past_asos = spring_wind_by_region("Bering Tundra", weather_stations, "past_asos").mean(axis=1)
# BTu_future_ncar = spring_wind_by_region("Bering Tundra", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# BTu_past_interim = spring_wind_by_region("Bering Tundra", weather_stations, "past_wrf_era_interim").mean(axis=1)
# BTu_future_gfdl_cm3 = spring_wind_by_region("Bering Tundra", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [19]:
BTu_past_asos

0     1127.00
1     1059.25
2     1077.25
3     1161.00
4      868.80
5      746.20
6      735.20
7      727.80
8      716.00
9      671.60
10     672.60
11     842.00
12     880.00
13     787.40
14     817.00
15     865.00
16    1001.40
17     848.00
18     881.60
19     811.00
20     827.00
21     807.80
22     988.00
23     959.40
24     973.80
25     964.00
26     919.60
27     827.00
28     810.80
29     840.60
30     884.60
31     826.80
32     744.60
33     960.00
34     765.60
dtype: float64

In [20]:
BTu_past_interim

0     869.0
1     903.2
2     895.2
3     922.8
4     830.2
5     918.6
6     853.2
7     908.6
8     872.0
9     919.4
10    823.4
11    821.6
12    862.0
13    891.6
14    852.6
15    882.4
16    939.4
17    901.0
18    896.8
19    779.4
20    848.8
21    914.0
22    921.8
23    852.6
24    857.0
25    852.4
26    864.0
27    773.6
28    750.8
29    852.0
30    822.0
31    787.2
32    880.2
33    929.8
34    918.0
dtype: float64

In [21]:
BTu_future_ncar[0:50]

0      905.8
1      838.8
2      828.8
3      894.4
4      830.4
5      901.0
6      847.6
7      813.8
8      950.0
9      871.2
10     779.4
11     838.8
12     812.8
13     860.0
14     901.4
15     853.8
16     935.4
17     931.8
18     909.2
19    1044.4
20     861.6
21     782.6
22     787.4
23     922.8
24     922.4
25     989.0
26     923.0
27     973.6
28     764.0
29     920.0
30     921.4
31     909.8
32    1020.0
33     826.4
34     947.6
35    1092.4
36     927.4
37     809.4
38     876.6
39     792.0
40     840.6
41     904.2
42    1140.2
43     825.2
44     855.6
45     898.2
46     831.6
47     932.4
48     849.0
49     882.0
dtype: float64

In [22]:
BTu_future_ncar[50:]

50     709.6
51     872.8
52    1002.0
53     832.4
54     825.2
55     959.6
56     946.4
57     800.0
58     846.4
59     893.2
60     867.6
61     875.6
62     801.4
63     871.0
64     942.8
65     831.2
66     956.0
67     842.8
68     924.4
69     711.8
70     934.8
71     957.0
72     841.6
73     949.6
74     844.4
75     863.0
76     905.8
77     872.2
78     826.6
79     886.4
80     960.2
81     780.6
82     924.2
83     927.8
84     900.8
dtype: float64

In [23]:
BTu_future_gfdl_cm3[0:50]

0     923.2
1     804.6
2     913.8
3     856.4
4     808.4
5     847.8
6     896.6
7     892.8
8     801.2
9     692.8
10    815.6
11    808.6
12    964.6
13    920.4
14    900.4
15    839.8
16    858.0
17    861.2
18    940.6
19    939.8
20    801.2
21    884.4
22    857.6
23    852.4
24    911.2
25    813.6
26    884.8
27    846.0
28    969.4
29    878.4
30    851.6
31    761.0
32    912.4
33    886.8
34    972.2
35    804.8
36    856.4
37    956.8
38    853.0
39    869.4
40    879.8
41    924.4
42    939.4
43    833.6
44    836.6
45    826.2
46    908.2
47    893.6
48    867.8
49    972.8
dtype: float64

In [24]:
BTu_future_gfdl_cm3[50:]

50    830.0
51    894.2
52    810.0
53    830.4
54    935.8
55    901.6
56    906.6
57    859.6
58    846.8
59    903.2
60    933.6
61    897.0
62    929.6
63    901.0
64    965.6
65    913.2
66    868.2
67    825.4
68    851.2
69    788.2
70    874.0
71    918.2
72    996.8
73    946.4
74    916.0
75    812.6
76    972.6
77    806.4
78    915.4
79    888.4
80    822.0
81    938.4
82    993.0
83    844.2
84    924.2
dtype: float64

In [25]:
# #Coastal Rainforests

# CR_past_asos = spring_wind_by_region("Coastal Rainforests", weather_stations, "past_asos").mean(axis=1)
# CR_future_ncar = spring_wind_by_region("Coastal Rainforests", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# CR_past_interim = spring_wind_by_region("Coastal Rainforests", weather_stations, "past_wrf_era_interim").mean(axis=1)
# CR_future_gfdl_cm3 = spring_wind_by_region("Coastal Rainforests", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [26]:
CR_past_asos

0     395.142857
1     348.666667
2     359.133333
3     366.500000
4     376.928571
5     385.642857
6     379.142857
7     424.357143
8     345.733333
9     340.733333
10    292.133333
11    381.933333
12    424.142857
13    348.466667
14    369.000000
15    316.000000
16    309.933333
17    390.333333
18    404.733333
19    439.733333
20    381.266667
21    458.133333
22    449.533333
23    428.800000
24    362.333333
25    372.733333
26    436.266667
27    377.600000
28    429.666667
29    354.800000
30    393.000000
31    354.466667
32    443.066667
33    383.333333
34    407.400000
dtype: float64

In [27]:
CR_past_interim

0     364.714286
1     318.000000
2     361.285714
3     371.250000
4     322.733333
5     396.142857
6     323.133333
7     440.071429
8     340.142857
9     314.466667
10    319.076923
11    343.733333
12    370.000000
13    305.800000
14    370.933333
15    344.153846
16    317.733333
17    347.769231
18    338.285714
19    361.400000
20    315.928571
21    348.733333
22    372.600000
23    342.533333
24    333.538462
25    333.769231
26    329.466667
27    304.000000
28    386.714286
29    303.714286
30    358.785714
31    294.285714
32    363.200000
33    311.428571
34    339.266667
dtype: float64

In [28]:
CR_future_ncar[0:50]

0     336.200000
1     307.928571
2     355.923077
3     340.857143
4     288.923077
5     305.428571
6     305.533333
7     295.285714
8     365.733333
9     288.733333
10    331.083333
11    352.428571
12    320.666667
13    313.466667
14    276.066667
15    322.400000
16    336.642857
17    291.466667
18    382.000000
19    332.857143
20    311.923077
21    393.833333
22    323.666667
23    370.200000
24    363.357143
25    293.266667
26    366.133333
27    356.200000
28    392.416667
29    341.533333
30    303.866667
31    315.466667
32    316.133333
33    299.785714
34    279.285714
35    301.000000
36    304.214286
37    294.533333
38    325.857143
39    331.642857
40    356.214286
41    297.615385
42    322.714286
43    324.533333
44    297.785714
45    340.642857
46    309.357143
47    305.714286
48    312.200000
49    295.357143
dtype: float64

In [29]:
CR_future_ncar[50:]

50    304.153846
51    354.272727
52    258.214286
53    284.571429
54    281.333333
55    308.066667
56    306.285714
57    255.533333
58    289.357143
59    338.133333
60    323.733333
61    288.285714
62    307.000000
63    347.733333
64    294.200000
65    350.142857
66    348.923077
67    309.357143
68    303.642857
69    282.333333
70    300.857143
71    275.071429
72    315.666667
73    300.600000
74    285.066667
75    301.466667
76    318.357143
77    361.307692
78    296.285714
79    278.500000
80    304.533333
81    287.066667
82    314.133333
83    301.800000
84    354.071429
dtype: float64

In [30]:
CR_future_gfdl_cm3[0:50]

0     361.928571
1     343.000000
2     313.214286
3     306.066667
4     334.416667
5     326.133333
6     355.714286
7     360.500000
8     295.133333
9     295.933333
10    344.800000
11    334.916667
12    333.833333
13    312.666667
14    346.923077
15    287.933333
16    292.071429
17    271.600000
18    340.000000
19    365.800000
20    369.090909
21    277.428571
22    264.928571
23    350.500000
24    306.857143
25    359.166667
26    271.714286
27    325.857143
28    320.214286
29    287.500000
30    332.928571
31    314.769231
32    298.357143
33    329.384615
34    311.142857
35    278.933333
36    264.714286
37    293.133333
38    300.384615
39    316.071429
40    292.142857
41    321.153846
42    282.466667
43    255.933333
44    274.933333
45    251.066667
46    269.461538
47    258.500000
48    285.538462
49    309.357143
dtype: float64

In [31]:
CR_future_gfdl_cm3[50:]

50    298.083333
51    291.133333
52    246.214286
53    295.666667
54    307.428571
55    296.692308
56    278.214286
57    323.333333
58    313.357143
59    281.333333
60    280.928571
61    310.076923
62    272.615385
63    313.846154
64    297.583333
65    302.142857
66    299.357143
67    218.928571
68    268.266667
69    251.333333
70    291.600000
71    286.857143
72    274.785714
73    279.533333
74    290.857143
75    271.800000
76    284.714286
77    261.000000
78    282.928571
79    268.357143
80    292.307692
81    312.384615
82    272.933333
83    257.916667
84    268.714286
dtype: float64

In [32]:
# #Copper River & Wrangell Mountains

# CRWM_past_asos = spring_wind_by_region("Copper River & Wrangell Mountains", weather_stations, "past_asos").mean(axis=1)
# CRWM_future_ncar = spring_wind_by_region("Copper River & Wrangell Mountains", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# CRWM_past_interim = spring_wind_by_region("Copper River & Wrangell Mountains", weather_stations, "past_wrf_era_interim").mean(axis=1)
# CRWM_future_gfdl_cm3 = spring_wind_by_region("Copper River & Wrangell Mountains", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [33]:
CRWM_past_asos

0     641.000000
1     609.250000
2     629.750000
3     785.333333
4     572.250000
5     623.250000
6     643.750000
7     688.500000
8     620.250000
9     625.750000
10    611.250000
11    619.000000
12    635.000000
13    616.750000
14    664.500000
15    635.750000
16    629.750000
17    653.500000
18    678.000000
19    639.750000
20    613.000000
21    651.750000
22    670.000000
23    664.250000
24    611.500000
25    605.000000
26    674.750000
27    631.250000
28    631.250000
29    612.000000
30    621.000000
31    615.000000
32    798.666667
33    789.666667
34    856.000000
dtype: float64

In [34]:
CRWM_past_interim

0     869.333333
1     805.000000
2     815.000000
3     793.000000
4     746.333333
5     837.666667
6     609.250000
7     637.750000
8     796.333333
9     822.000000
10    772.333333
11    580.250000
12    639.250000
13    767.000000
14    880.666667
15    832.333333
16    612.500000
17    816.333333
18    852.000000
19    615.500000
20    597.500000
21    591.500000
22    851.333333
23    810.333333
24    793.333333
25    754.333333
26    642.750000
27    826.333333
28    835.333333
29    828.666667
30    844.666667
31    661.750000
32    846.333333
33    765.000000
34    680.000000
dtype: float64

In [35]:
CRWM_future_ncar[0:50]

0     703.000000
1     742.000000
2     782.333333
3     735.333333
4     742.000000
5     679.500000
6     763.333333
7     765.666667
8     766.666667
9     763.666667
10    771.333333
11    698.666667
12    586.750000
13    728.666667
14    705.666667
15    774.000000
16    560.250000
17    755.000000
18    586.500000
19    772.666667
20    760.666667
21    727.333333
22    712.333333
23    621.000000
24    583.250000
25    755.333333
26    840.666667
27    813.000000
28    723.000000
29    642.000000
30    767.666667
31    782.333333
32    766.333333
33    717.000000
34    750.666667
35    795.000000
36    599.250000
37    755.666667
38    600.250000
39    789.666667
40    584.500000
41    731.666667
42    858.000000
43    759.666667
44    726.666667
45    797.666667
46    822.666667
47    839.000000
48    798.000000
49    771.666667
dtype: float64

In [36]:
CRWM_future_ncar[50:]

50    761.333333
51    759.000000
52    758.000000
53    769.333333
54    591.500000
55    566.500000
56    782.333333
57    766.666667
58    564.500000
59    895.000000
60    578.500000
61    587.250000
62    572.750000
63    850.000000
64    772.000000
65    773.000000
66    765.000000
67    576.250000
68    829.666667
69    747.333333
70    767.666667
71    822.666667
72    602.750000
73    829.333333
74    584.500000
75    560.000000
76    798.666667
77    875.666667
78    563.750000
79    851.333333
80    801.000000
81    745.333333
82    724.666667
83    825.000000
84    801.666667
dtype: float64

In [37]:
CRWM_future_gfdl_cm3[0:50]

0     730.000000
1     704.666667
2     751.666667
3     570.250000
4     683.333333
5     599.000000
6     609.000000
7     551.000000
8     740.666667
9     550.000000
10    742.333333
11    731.000000
12    744.666667
13    575.000000
14    759.000000
15    581.250000
16    559.500000
17    781.000000
18    599.500000
19    587.500000
20    712.000000
21    573.000000
22    754.000000
23    817.000000
24    786.666667
25    728.666667
26    766.000000
27    743.000000
28    824.333333
29    795.666667
30    790.000000
31    739.333333
32    572.500000
33    790.333333
34    680.750000
35    761.333333
36    767.333333
37    605.000000
38    587.000000
39    744.000000
40    725.333333
41    790.333333
42    793.666667
43    777.333333
44    794.666667
45    589.750000
46    771.666667
47    593.750000
48    779.333333
49    613.500000
dtype: float64

In [38]:
CRWM_future_gfdl_cm3[50:]

50    761.333333
51    849.333333
52    601.000000
53    604.500000
54    804.666667
55    568.000000
56    831.000000
57    781.666667
58    775.000000
59    793.666667
60    807.333333
61    804.000000
62    793.333333
63    805.333333
64    836.666667
65    793.333333
66    797.666667
67    775.333333
68    659.500000
69    576.500000
70    598.750000
71    824.000000
72    781.333333
73    610.750000
74    852.000000
75    587.250000
76    642.250000
77    800.000000
78    855.000000
79    572.000000
80    785.000000
81    773.000000
82    823.000000
83    774.333333
84    838.333333
dtype: float64

In [ ]:
#NORTH SLOPE

#NS_past_asos = spring_wind_by_region("North Slope", weather_stations, "past_asos").mean(axis=1)
#NS_future_ncar = spring_wind_by_region("North Slope", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
#NS_past_interim = spring_wind_by_region("North Slope", weather_stations, "past_wrf_era_interim").mean(axis=1)
#NS_future_gfdl_cm3 = spring_wind_by_region("North Slope", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)